# Import the relevant packages


In [6]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

# Data

In [7]:
mnist_dataset,mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [8]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [11]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,dtype=tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,dtype=tf.int64)


def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [12]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Model
Outline the model

In [15]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

# Choose the optimizer and the loss function

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training

In [17]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)


Epoch 1/5
540/540 - 5s - loss: 0.4703 - accuracy: 0.8793 - val_loss: 0.2293 - val_accuracy: 0.9350
Epoch 2/5
540/540 - 3s - loss: 0.1791 - accuracy: 0.9475 - val_loss: 0.1474 - val_accuracy: 0.9575
Epoch 3/5
540/540 - 3s - loss: 0.1260 - accuracy: 0.9620 - val_loss: 0.1143 - val_accuracy: 0.9678
Epoch 4/5
540/540 - 3s - loss: 0.0952 - accuracy: 0.9721 - val_loss: 0.0896 - val_accuracy: 0.9747
Epoch 5/5
540/540 - 3s - loss: 0.0758 - accuracy: 0.9776 - val_loss: 0.0807 - val_accuracy: 0.9768


# Test the model

In [18]:
test_data, test_accuracy = model.evaluate(test_data)
print(test_data, test_accuracy)

1/1 [==============================] - 0s 1ms/step - loss: 0.0899 - accuracy: 0.9719
0.08989302068948746 0.9718999862670898
